In [ ]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import Vectorizer, HeadQA, HeadQA_IR, clean_words, parse_dataset, parse_ir_dataset, random_oversamplig, save_dataset_to_pickle, load_dataset_from_pickle
from training import train, validate, evaluate, train_ir, validate_ir

import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

In [ ]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [ ]:
#training_instances = parse_ir_dataset(training)
#validation_instances = parse_ir_dataset(validation)
#testing_instances = parse_ir_dataset(testing)

#oversampled_training = random_oversamplig(training_instances)

#save_dataset_to_pickle('../data/training_ir.pickle', training_instances)
#save_dataset_to_pickle('../data/validation_ir.pickle', validation_instances)
#save_dataset_to_pickle('../data/testing_ir.pickle', testing_instances)
#save_dataset_to_pickle('../data/oversampled_training_ir.pickle', oversampled_training)

In [ ]:
training_instances = load_dataset_from_pickle('../data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('../data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('../data/testing_ir.pickle')
oversampled_training = load_dataset_from_pickle('../data/oversampled_training_ir.pickle')

In [ ]:
oversampled_training[0]

In [ ]:
vectorizer = Vectorizer.vectorize_ir_dataset(oversampled_training)

In [ ]:
vocab = vectorizer.sentence_vocab
label_vocab = vectorizer.label_vocab

In [ ]:
trainset = HeadQA_IR(instances=training_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
validset = HeadQA_IR(instances=validation_instances, vectorizer=vectorizer, right_padding=False, max_length=15)
testset = HeadQA_IR(instances=testing_instances, vectorizer=vectorizer, right_padding=False, max_length=15)

In [ ]:
trainset.instances[0]

In [ ]:
batch_size = 32
train_dt = DataLoader(trainset, batch_size=batch_size,drop_last=True)
valid_dt = DataLoader(validset, batch_size=batch_size,drop_last=True)
test_dt = DataLoader(testset, batch_size=batch_size,drop_last=True)

In [ ]:
class LSTM_QA(torch.nn.Module):
    def __init__(self, num_embeddings, vocab_size, hidden_dim, x_size, n_classes, embedding_dim=300,
                pretrained_embeddings=None): 
        super(LSTM_QA, self).__init__()
        self.lstm_0 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=0.5, bidirectional=True)
        self.lstm_1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, dropout=0.5, bidirectional=True)
        #nn.CosineSimilarity
        self.linear = nn.Linear(hidden_dim, n_classes)
        self.dropout = nn.Dropout(0.5)
        self.n_classes = n_classes
        
    def forward(self, x_0, x_1):
        x_0 = torch.unsqueeze(x_0, dim=2, ).long()
        x_1 = torch.unsqueeze(x_1, dim=2).long()
        out_0, (ht_0, ct_0) = self.lstm_0(x_0)
        out_1, (ht_1, ct_1) = self.lstm_0(x_1)
        print(out_0.shape, out_1.shape)
        print(ht_0.shape, ht_1.shape)
        print(ct_0,shape, ct_1.shape)
        return 0  

In [ ]:
def get_optimizer(model, lr=0.01, wd=0.0):
    return torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)

In [ ]:
model = LSTM_QA(50, len(vocab), hidden_dim=64, x_size=trainset.max_length, n_classes=1, embedding_dim=1)
optimizer = get_optimizer(model, lr = 0.001, wd = 1e-5)

In [ ]:
def train_ir(model, optimizer, train_dl, test_dl, validate, epochs=50):
    y_trues, y_preds = [], []
    epochs_results = []
    for i in range(epochs):
        model.train()
        total = 0
        sum_loss = 0
        for x_0, x_1, y in train_dl:
            batch = y.shape[0]
            out = model(x_0.long(), x_1.long())
            loss = F.binary_cross_entropy(out, y.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total += batch
            sum_loss += batch*(loss.item())
        train_loss = sum_loss/total
        valid_loss, valid_acc, y_real, y_pred = validate(model, test_dl)
        y_trues.append(y_real)
        y_preds.append(y_pred)
        epochs_results.append([train_loss, valid_loss, valid_acc])
        print("Epoch %s train loss  %.4f valid loss %.3f and accuracy %.4f" %
              (i, train_loss, valid_loss, valid_acc))
    return epochs_results

In [ ]:
training_results = train_ir(model, optimizer, train_dt, valid_dt, validate, epochs=1)